# **Homework 2 Phoneme Classification**

* Slides: https://docs.google.com/presentation/d/1v6HkBWiJb8WNDcJ9_-2kwVstxUWml87b9CnA16Gdoio/edit?usp=sharing
* Kaggle: https://www.kaggle.com/c/ml2022spring-hw2
* Video: TBA


In [2]:
# !nvidia-smi

## Download Data
Download data from google drive, then unzip it.

You should have
- `libriphone/train_split.txt`
- `libriphone/train_labels`
- `libriphone/test_split.txt`
- `libriphone/feat/train/*.pt`: training feature<br>
- `libriphone/feat/test/*.pt`:  testing feature<br>

after running the following block.

> **Notes: if the google drive link is dead, you can download the data directly from [Kaggle](https://www.kaggle.com/c/ml2022spring-hw2/data) and upload it to the workspace**


### Download train/test metadata

In [3]:
# !pip install --upgrade gdown

# Main link
# !gdown --id '1o6Ag-G3qItSmYhTheX6DYiuyNzWyHyTc' --output libriphone.zip

# Backup link 1
# !gdown --id '1R1uQYi4QpX0tBfUWt2mbZcncdBsJkxeW' --output libriphone.zip

# Bqckup link 2
# !wget -O libriphone.zip "https://www.dropbox.com/s/wqww8c5dbrl2ka9/libriphone.zip?dl=1"

# !unzip -q libriphone.zip
# !ls libriphone

### Preparing Data

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [1]:
import os
import random
import pandas as pd
import torch
from tqdm import tqdm

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]

    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n) 
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, train_val_seed=1337):
    class_num = 41 # NOTE: pre-computed, should not need change
    mode = 'train' if (split == 'train' or split == 'val') else 'test'

    label_dict = {}
    if mode != 'test':
      phone_file = open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines()

      for line in phone_file:
          line = line.strip('\n').split(' ')
          label_dict[line[0]] = [int(p) for p in line[1:]]

    if split == 'train' or split == 'val':
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(train_val_seed)
        random.shuffle(usage_list)
        percent = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:percent] if split == 'train' else usage_list[percent:]
    elif split == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)
    if mode != 'test':
      y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode != 'test':
          label = torch.LongTensor(label_dict[fname])

        X[idx: idx + cur_len, :] = feat
        if mode != 'test':
          y[idx: idx + cur_len] = label

        idx += cur_len

    X = X[:idx, :]
    if mode != 'test':
      y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode != 'test':
      print(y.shape)
      return X, y
    else:
      return X


## Define Dataset

In [11]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


## Define Model

In [2]:
'''2DCNN-GRU'''
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import GRUCell

class Classifier1(nn.Module):
    def __init__(self, input_dim, output_dim = 41, hidden_layers=3, hidden_dim=256, concat_nframes=31, batch_size=512):
        super(Classifier1, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.hidden_layers = hidden_layers
        self.batch_size=batch_size
        self.concat_nframes = concat_nframes
        
        self.CN1 = nn.Conv2d(1,32,(3,3),stride=(1, 2),padding = 0)
        self.CN2 = nn.Conv2d(32,32,(3,3),stride=(1, 1),padding = 0)
#         self.CN3 = nn.Conv2d(32,32,(3,3),stride=(1, 1),padding = 0)
        
        
        self.gru = nn.GRU(128, 128, self.hidden_layers, batch_first=True, dropout=0.5)
        
        self.fc = nn.Linear(17*32,128)
        self.MP = nn.MaxPool2d((1, 2),stride =(1,2) )
        self.LN1 = nn.LayerNorm(19)
        self.LN2 = nn.LayerNorm(17)
        self.LN3 = nn.LayerNorm(15)
        
        self.BN1 = nn.BatchNorm1d((self.concat_nframes-4)*self.hidden_dim)
        self.BN2 = nn.BatchNorm1d(2048)
        self.BN3 = nn.BatchNorm1d(512)
        
        self.LI1 = nn.Linear((self.concat_nframes-4)*self.hidden_dim, 2048)
        self.LI2 = nn.Linear(2048, 512)
        
        self.out = nn.Linear(512, output_dim)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x ,h):
        if h is None:
            h = torch.autograd.Variable(torch.zeros(1 * self.hidden_layers, batch_size, self.hidden_dim )).to(device)
        if self.batch_size != x.size(0):
            self.batch_size = x.size(0)
            h = torch.index_select(h,1,torch.tensor(range(self.batch_size)).to(device))
        x = x.view(self.batch_size,1 , self.concat_nframes, self.input_dim)
        x = self.CN1(x)
        x = self.LN1(x)
        x = F.gelu(x)
        x = self.dropout(x)
        x = self.CN2(x)
        x = self.LN2(x)
        x = F.gelu(x)
        x = self.dropout(x)
#         x = self.CN3(x)
#         x = self.LN3(x)
#         x = F.gelu(x)
#         x = self.dropout(x)
        
        x = x.transpose(2,3).reshape(self.batch_size, -1, self.concat_nframes-4)
        x = self.fc(x.transpose(1,2))     # batch sequence dim
            
        x, h = self.gru(x, h)
        x = F.gelu(x)
        x = self.dropout(x)
        
        x = self.BN1(x.reshape(self.batch_size,-1))
        x = self.LI1(x)
        x = F.gelu(x)
        x = self.dropout(x)
        
        x = self.BN2(x)
        x = self.LI2(x)
        x = F.gelu(x)
        x = self.dropout(x)
        
        x = self.BN3(x)
        x = self.out(x)
        return x, h
    
    def init_hidden(self, batch_size):
        return torch.autograd.Variable(torch.zeros(1 * self.hidden_layers, batch_size, self.hidden_dim )).to(device)

In [3]:
'''GRU'''
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import GRUCell

class Classifier2(nn.Module):
    def __init__(self, input_dim, output_dim = 41, hidden_layers=3, hidden_dim=256, concat_nframes=31, batch_size=512):
        super(Classifier2, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.hidden_layers = hidden_layers
        self.batch_size=batch_size
        self.hidden_dim2 = hidden_dim*concat_nframes
        
        self.gru = nn.GRU(self.input_dim, self.hidden_dim, self.hidden_layers, bidirectional = True, batch_first=True, dropout=0.4)
        
        self.BN1 = nn.BatchNorm1d(self.input_dim)
        self.BN2 = nn.BatchNorm1d(self.hidden_dim2*2)
        self.BN3 = nn.BatchNorm1d(2048)
        self.BN4 = nn.BatchNorm1d(512)
        
        self.LI1 = nn.Linear(self.hidden_dim2*2, 2048)
        self.LI2 = nn.Linear(2048, 512)
        self.LI3 = nn.Linear(1024, 512)
        
        self.out = nn.Linear(512, output_dim)
        
        self.relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x, h):
        
        if h is None:
            h = torch.autograd.Variable(torch.zeros(2 * self.hidden_layers, batch_size, self.hidden_dim )).to(device)
        if self.batch_size != x.size(0):
            self.batch_size = x.size(0)
            h = torch.index_select(h,1,torch.tensor(range(self.batch_size)).to(device))
            
        x, h = self.gru(x.view(self.batch_size, concat_nframes, input_dim), h)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.BN2(x.reshape(self.batch_size,-1))
        x = self.LI1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.BN3(x)
        x = self.LI2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
#         x = self.BN4(x)
#         x = self.LI3(x)
#         x = self.relu(x)
#         x = self.dropout(x)
        x = self.BN4(x)
        x = self.out(x)
        return x, h
    
    def init_hidden(self, batch_size):
        return torch.autograd.Variable(torch.zeros(2 * self.hidden_layers, batch_size, self.hidden_dim )).to(device)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F



class Classifier3(nn.Module):
    def __init__(self, input_dim, output_dim=41):
        super(Classifier3, self).__init__()

        self.fc = nn.Sequential(
            nn.Linear(input_dim, output_dim),
        )

        

    def forward(self, x):
        x = self.fc(x)
        return x

## Hyper-parameters

In [5]:
# data prarameters
concat_nframes = 101             # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 1.0               # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 0                        # random seed
batch_size = 512                # batch size
num_epoch = 10                  # the number of training epoch
learning_rate = 0.001           # learning rate
model_path1 = './1DCNNGRU.ckpt'     # the path where the checkpoint will be saved
model_path2 = './GRU.ckpt'     # the path where the checkpoint will be saved
model_path3 = './ensemble.ckpt'     # the path where the checkpoint will be saved

# model parameters
input_dim = 39                  # the input dim of the model, you should not change the value
hidden_layers1 = 4               # the number of hidden layers
hidden_layers2 = 6               # the number of hidden layers
hidden_dim = 128               # the hidden dim
output_dim = 41

## Prepare dataset and model

In [7]:
import gc

# preprocess data
train_X, train_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)
val_X, val_y = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)

# get dataset
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for train: 4286


4286it [00:43, 98.88it/s] 


[INFO] train set
torch.Size([2644158, 3939])
torch.Size([2644158])
[Dataset] - # phone classes: 41, number of utterances for val: 0


0it [00:00, ?it/s]


[INFO] val set
torch.Size([0, 3939])
torch.Size([0])


In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

DEVICE: cuda


In [8]:
import numpy as np

#fix seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [9]:
# fix random seed
same_seeds(seed)

# create model, define a loss function, and optimizer
model1 = Classifier1(input_dim=input_dim, hidden_layers=hidden_layers1, hidden_dim=hidden_dim, concat_nframes=concat_nframes).to(device)
model2 = Classifier2(input_dim=input_dim, hidden_layers=hidden_layers2, hidden_dim=hidden_dim, concat_nframes=concat_nframes).to(device)
model3 = Classifier3(input_dim=2*output_dim).to(device)
criterion = nn.CrossEntropyLoss() 
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, nesterov = True, momentum=0.7)
optimizer1 = torch.optim.AdamW(model1.parameters(), betas=(0.9, 0.98), lr=learning_rate, weight_decay=0.01)
optimizer12 = torch.optim.SGD(model1.parameters(), lr=learning_rate, momentum=0.9)
optimizer2 = torch.optim.AdamW(model2.parameters(), betas=(0.9, 0.98), lr=learning_rate, weight_decay=0.01)
optimizer22 = torch.optim.SGD(model2.parameters(), lr=learning_rate, momentum=0.9)
# optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
optimizer3 = torch.optim.AdamW(model3.parameters(), betas=(0.9, 0.98), lr=learning_rate, weight_decay=0.01)
optimizer32 = torch.optim.SGD(model3.parameters(), lr=learning_rate, momentum=0.9)

## Training

In [11]:
'''這是2DCNNGRU'''
best_acc = 0.0
early_stop = 0
optimizer_switch = True
model1.load_state_dict(torch.load(model_path1))
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    
    # training
    model1.train() # set the model to training mode
#     if epoch > 6:
#         optimizer1 = optimizer12
    
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)
        
        optimizer1.zero_grad() 
        outputs, h = model1(features, None) #h.detach()) 
        
        loss = criterion(outputs, labels)
        loss.backward() 
        optimizer1.step() 
        
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()
    
    # validation
    if len(val_set) > 0:
        model1.eval() # set the model to evaluation mode

        with torch.no_grad():
            for i, batch in enumerate(tqdm(val_loader)):
                features, labels = batch
                features = features.to(device)
                labels = labels.to(device)
                
                outputs, h = model1(features, None)
                loss = criterion(outputs, labels) 
                
                _, val_pred = torch.max(outputs, 1) 
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                early_stop = 0
                best_acc = val_acc
                torch.save(model1.state_dict(), model_path1)
                print('saving model1 with acc {:.3f}'.format(best_acc/len(val_set)))
            else:
                early_stop += 1
                print(early_stop)
            
            if early_stop > 1 and optimizer_switch == True:
                print('switch optimizer')
                optimizer1 = optimizer12
                optimizer_switch = False
                early_stop = 0
            elif early_stop > 2 and optimizer_switch == False:
                print('early stop!')
                break
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model1.state_dict(), model_path1)
    print('saving model at last epoch')


100%|██████████| 5165/5165 [13:15<00:00,  6.49it/s]


[001/010] Train Acc: 0.768966 Loss: 0.744105


100%|██████████| 5165/5165 [13:27<00:00,  6.40it/s]


[002/010] Train Acc: 0.785271 Loss: 0.683989


100%|██████████| 5165/5165 [13:33<00:00,  6.35it/s]


...

[008/010] Train Acc: 0.836777 Loss: 0.500726


100%|██████████| 5165/5165 [13:09<00:00,  6.54it/s]


[009/010] Train Acc: 0.840870 Loss: 0.486197


100%|██████████| 5165/5165 [13:02<00:00,  6.60it/s]


[010/010] Train Acc: 0.842643 Loss: 0.480316
saving model at last epoch


In [12]:
'''這是GRU'''
best_acc = 0.0
early_stop = 0
optimizer_switch = True
model2.load_state_dict(torch.load(model_path2))
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    
#     if epoch > 6:
#         optimizer2 = optimizer22
    # training
    model2.train() # set the model to training mode
    
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)
        
        optimizer22.zero_grad() 
        outputs, h = model2(features, None) #h.detach()) 
        
        loss = criterion(outputs, labels)
        loss.backward() 
        optimizer22.step() 
        
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()
    
    # validation
    if len(val_set) > 0:
        model2.eval() # set the model to evaluation mode
        h = model2.init_hidden(batch_size)
        with torch.no_grad():
            for i, batch in enumerate(tqdm(val_loader)):
                features, labels = batch
                features = features.to(device)
                labels = labels.to(device)
                
                outputs, h = model2(features, h.detach())
                loss = criterion(outputs, labels) 
                
                _, val_pred = torch.max(outputs, 1) 
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                early_stop = 0
                best_acc = val_acc
                torch.save(model2.state_dict(), model_path2)
                print('saving model2 with acc {:.3f}'.format(best_acc/len(val_set)))
            else:
                early_stop += 1
                print(early_stop)
            
            if early_stop > 1 and optimizer_switch == True:
                print('switch optimizer')
                optimizer2 = optimizer22
                optimizer_switch = False
                early_stop = 0
            elif early_stop > 2 and optimizer_switch == False:
                print('early stop!')
                break
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model2.state_dict(), model_path2)
    print('saving model2 at last epoch')


100%|██████████| 5165/5165 [11:19<00:00,  7.60it/s]


[001/010] Train Acc: 0.911403 Loss: 0.309911


100%|██████████| 5165/5165 [11:20<00:00,  7.59it/s]


[002/010] Train Acc: 0.912603 Loss: 0.292936


100%|██████████| 5165/5165 [11:25<00:00,  7.53it/s]


...

[008/010] Train Acc: 0.916409 Loss: 0.264278


100%|██████████| 5165/5165 [11:34<00:00,  7.44it/s]


[009/010] Train Acc: 0.916589 Loss: 0.261414


100%|██████████| 5165/5165 [11:33<00:00,  7.45it/s]


[010/010] Train Acc: 0.917273 Loss: 0.259704
saving model2 at last epoch


In [13]:
'''這是ensemble'''
best_acc = 0.0
early_stop = 0
optimizer_switch = True

model1.load_state_dict(torch.load(model_path1))
model2.load_state_dict(torch.load(model_path2))
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
#     if epoch > 6:
#         optimizer3 = optimizer32

    # training
    model1.eval() # set the model to evaluation mode
    model2.eval() # set the model to evaluation mode
    model3.train() # set the model to training mode

    for i, batch in enumerate(tqdm(train_loader)):
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)
        with torch.no_grad(): 
            outputs1, h1 = model1(features, None)
            outputs2, h2 = model2(features, None)
            features2 = torch.cat((outputs1,outputs2),1)
        
        
        optimizer3.zero_grad() 
        outputs = model3(features2) 
        
        loss = criterion(outputs, labels)
        loss.backward() 
        optimizer3.step() 
        
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()
    
    # validation
    if len(val_set) > 0:
        model1.eval() # set the model to evaluation mode
        model2.eval() # set the model to evaluation mode
        model3.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, batch in enumerate(tqdm(val_loader)):
                features, labels = batch
                features = features.to(device)
                labels = labels.to(device)
                outputs1, h1 = model1(features, None)
                outputs2, h2 = model2(features, None)
#                 outputs1 = SM(outputs1)
#                 outputs2 = SM(outputs2)
                features2 = torch.cat((outputs1,outputs2),dim = 1)
                
                outputs = model3(features2) 
                loss = criterion(outputs, labels) 
                
                _, val_pred = torch.max(outputs, 1) 
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                early_stop = 0
                best_acc = val_acc
                torch.save(model3.state_dict(), model_path3)
                print('saving model3 with acc {:.3f}'.format(best_acc/len(val_set)))
            else:
                early_stop += 1
                print(early_stop)
            
            if early_stop > 1 and optimizer_switch == True:
                print('switch optimizer')
                optimizer3 = optimizer32
                optimizer_switch = False
                early_stop = 0
            elif early_stop > 2 and optimizer_switch == False:
                print('early stop!')
                break
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model3.state_dict(), model_path3)
    print('saving model3 at last epoch')


100%|██████████| 5165/5165 [10:11<00:00,  8.44it/s]


[001/010] Train Acc: 0.929124 Loss: 0.230834


100%|██████████| 5165/5165 [10:11<00:00,  8.45it/s]


[002/010] Train Acc: 0.950290 Loss: 0.142373


100%|██████████| 5165/5165 [10:11<00:00,  8.45it/s]


...

[008/010] Train Acc: 0.957136 Loss: 0.129914


100%|██████████| 5165/5165 [10:10<00:00,  8.46it/s]


[009/010] Train Acc: 0.957377 Loss: 0.129489


100%|██████████| 5165/5165 [10:57<00:00,  7.86it/s]  

[010/010] Train Acc: 0.957575 Loss: 0.129348
saving model3 at last epoch


In [15]:
del train_loader, val_loader
gc.collect()

824

## Testing
Create a testing dataset, and load model from the saved checkpoint.

In [12]:
# load data
test_X = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 1078


1078it [00:11, 90.56it/s] 


[INFO] test set
torch.Size([646268, 3939])


In [13]:
# load model

model1 = Classifier1(input_dim=input_dim, hidden_layers=hidden_layers1, hidden_dim=hidden_dim, concat_nframes=concat_nframes).to(device)
model2 = Classifier2(input_dim=input_dim, hidden_layers=hidden_layers2, hidden_dim=hidden_dim, concat_nframes=concat_nframes).to(device)
model3 = Classifier3(input_dim=2*output_dim).to(device)
model1.load_state_dict(torch.load(model_path1))
model2.load_state_dict(torch.load(model_path2))
model3.load_state_dict(torch.load(model_path3))

<All keys matched successfully>

Make prediction.

In [14]:
test_acc = 0.0
test_lengths = 0
pred = np.array([], dtype=np.int32)
model1.eval()
model2.eval()
model3.eval()

with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        features = features.to(device)

        outputs1, h1 = model1(features, None)
        outputs2, h2 = model2(features, None)
#         outputs1 = SM(outputs1)
#         outputs2 = SM(outputs2)
        features2 = torch.cat((outputs1,outputs2),1)

        outputs = model3(features2) 
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


100%|██████████| 1263/1263 [02:26<00:00,  8.64it/s]


In [35]:
test_acc = 0.0
test_lengths = 0
pred = np.array([], dtype=np.int32)
model1.eval()

with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        features = features.to(device)

#         outputs, h1 = model1(features, None)
        outputs, h2 = model1(features, None)


        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


100%|██████████| 1263/1263 [01:40<00:00, 12.55it/s]


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [15]:
with open('prediction_ensemble3.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))